In [1]:
import warnings

import cantera as ct
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt

import sdtoolbox as sdt

In [2]:
gas = ct.Solution("gri30_highT.xml")

In [3]:
# keeping these separate from species because I don't want only the H reaction for now
rxns = []
rxn_indices = []
for i, rxn in enumerate(gas.reactions()):
    reactants_and_products = [*rxn.reactants.keys(),*rxn.products.keys()]
    if (
        "CO2" in reactants_and_products
        # or
        # "N2" in reactants_and_products
    ) and "OH" in reactants_and_products:
        rxns.append(rxn)
        rxn_indices.append(i)
rxns, rxn_indices

([<ElementaryReaction: CO + OH <=> CO2 + H>,
  <ElementaryReaction: CO + HO2 <=> CO2 + OH>,
  <ElementaryReaction: HNCO + OH <=> CO2 + NH2>],
 [98, 119, 267])

In [4]:
for rxn in rxns:
    if hasattr(rxn, "rate"):
        print(rxn, rxn.rate)
    else:
        print(rxn)

CO + OH <=> CO2 + H Arrhenius(A=47600, b=1.228, E=292880)
CO + HO2 <=> CO2 + OH Arrhenius(A=1.5e+11, b=0, E=9.87424e+07)
HNCO + OH <=> CO2 + NH2 Arrhenius(A=3300, b=1.5, E=1.50624e+07)


In [5]:
by_a = sorted(rxns, key=lambda r: r.rate.pre_exponential_factor, reverse=True)
by_a

[<ElementaryReaction: CO + HO2 <=> CO2 + OH>,
 <ElementaryReaction: CO + OH <=> CO2 + H>,
 <ElementaryReaction: HNCO + OH <=> CO2 + NH2>]

In [6]:
species = ["CO2", "OH", "H"]
species_indices = []
for spec in species:
    try:
        species_indices.append(gas.species_index(spec))
    except ValueError:
        warnings.warn(f"Species not found: {spec}")
species_indices

[15, 4, 1]

In [7]:
mech = "gri30_highT.xml"
fuel = "CH4"
oxidizer = "N2O"
diluent = None
t0 = 290
p0 = 101325
phi=1.0
dil_mf = 0
base_gas = ct.Solution(mech)
base_gas.set_equivalence_ratio(phi, fuel, oxidizer)

q = base_gas.mole_fraction_dict()

# CJ speed
dcj = sdt.postshock.CJspeed(p0, t0, q, mech)

# # post shock EQ
# working_gas = sdt.postshock.PostShock_eq(dcj, p0, t0, q, mech)
# dcj_density_corrected = dcj * base_gas.density / working_gas.density

# post shock FR
working_gas = sdt.postshock.PostShock_fr(dcj, p0, t0, q, mech)

In [8]:
data_dir = %pwd
db_location = f"{data_dir}/co2_sims.sqlite"
db_location

'/home/mick/DetResearch/scripts/final_manuscript/co2_sims.sqlite'

In [9]:
db = sdt.output.SimulationDatabase(
    db=sdt.output.SqliteDataBase(db_location),
    conditions=sdt.output.Conditions(
        sim_type=sdt.output.SimulationType.Znd.value,
        mech=mech,
        initial_temp=t0,
        initial_press=p0,
        fuel=fuel,
        oxidizer=oxidizer,
        equivalence=phi,
        diluent=diluent,
        dil_mf=dil_mf,
    ),
)

In [10]:
# znd ODEs
znd = sdt.znd.zndsolve(
    working_gas,
    base_gas,
    dcj,
    t_end=2e-3,
    max_step=1e-4,
    advanced_output=True,
    rxn_indices=rxn_indices,
    spec_indices=species_indices,
    db=db,
)

/home/mick/anaconda3/envs/research2/lib/python3.9/site-packages/scipy/integrate/_ivp/common.py:338: RuntimeWarning: overflow encountered in multiply
  new_factor = NUM_JAC_FACTOR_INCREASE * factor[ind]
/home/mick/anaconda3/envs/research2/lib/python3.9/site-packages/scipy/integrate/_ivp/common.py:360: RuntimeWarning: overflow encountered in multiply
  factor[max_diff < NUM_JAC_DIFF_SMALL * scale] *= NUM_JAC_FACTOR_INCREASE


In [ ]:
# CV parameters
working_gas.TPX = t0, p0, q
working_gas = sdt.postshock.PostShock_fr(dcj, p0, t0, q, mech)
t_vn, p_vn = working_gas.TP

# CV sims
t_a = t_vn * 1.02
working_gas.TPX = t_a, p_vn, q
limit_species_idx = working_gas.species_index(fuel)  # we're not running rich
cv_0 = sdt.cv.cvsolve(
    working_gas, 
    t_end=2*1e-6, 
    max_step=2*1e-5,
    rxn_indices=rxn_indices,
    spec_indices=species_indices,
    db=db,
)

In [ ]:
# t_b = t_vn * 0.98
# working_gas.TPX = t_b, p_vn, q
# cv_1 = sdt.cv.cvsolve(working_gas, t_end=2*2*1e-6, max_step=2*2*1e-5)

In [ ]:
# products
working_gas.TPX = t_vn, p_vn, q
working_gas.equilibrate("UV")
t_final = working_gas.T

# westbrook
t_50 = 0.5 * (t_final - t_vn) + t_vn  # 50% temp rise
ind_time = np.nanmax(cv_0["time"][cv_0["T"] < t_50], initial=0)
ind_len = ind_time * znd["U"][0]
cell_size = 29 * ind_len

print(cell_size*1000)

In [ ]:
start_interval = 40
end_interval = 100

In [ ]:
sns.lineplot(x=znd["time"][start_interval:end_interval], y=znd["thermicity"][start_interval:end_interval])
plt.xlabel("Time (sec)")
plt.ylabel("Thermicity")
plt.title("Thermicity Peak")
sns.despine()

In [ ]:
species = ["CO2", "OH", "H"]
for spec in species:
    plt.plot(znd["time"][start_interval:end_interval], znd["species"][gas.species_index(spec)][start_interval:end_interval])
    # plt.axvline(ind_time)
plt.legend(species)
plt.xlabel("Time (sec)")
plt.ylabel("Concentration")
# plt.ylabel(f"{spec} concentration")
# plt.title(spec)
sns.despine()

In [ ]:
znd["ind_len_ZND"], ind_len, znd["ind_len_ZND"]/  ind_len * 100

In [ ]:
znd.keys()

In [ ]:
znd["gas1"].reactions()[98]

In [ ]:
znd["gas1"].reaction_equation(98)

In [ ]:
gas.forward_rates_of_progress[rxn_indices]

In [ ]:
gas.forward_rate_constants[rxn_indices]

In [ ]:
test = np.array([[]])

In [ ]:
np.append(test, gas.forward_rates_of_progress[rxn_indices])

In [ ]:
cv_0.keys()

In [ ]:
sdt.utilities.cv_plot(cv_0)


In [ ]:
sdt.utilities.znd_plot(znd)